In [ ]:
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd

import awkward as ak
import awkward_pandas as akpd

from astropy.time import Time

from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.tmaUtils import TMAEventMaker

In [ ]:
# Use the blockUtils functions to get info on what blocks were used.
'''
For  

block_parser = BlockParser(day_obs)  

you can use:

block_parser.getBlockNums()  
block_parser.getSeqNums(227)  
block_parser.getRows(227,1)  

block_parser.printBlockEvolution(227)  
block_parser.getBlockInfo(227,6)  
block_parser.getEventsForBlock(events, 219,1)
'''

# Set the day_obs list 
day_obs_list = range(20240205, 20240205 + 11)

# For the TMA events 
event_maker = TMAEventMaker()

In [ ]:
# For each day_obs in the list determine which blocks were run and put
# the list of blocks into the block_list.
 
block_list = []

for day_obs in day_obs_list:
    block_parser = BlockParser(day_obs)
    blocks = block_parser.getBlockNums()
    block_list.append(blocks)

# Put the variable length nested list into an awkward array and then 
# put that into a pandas dataframe with the awkward array extension
# so that the list of blocks is shown in a column.
blocks = ak.Array({"day_obs": day_obs_list, "blocks": block_list})
series = akpd.from_awkward(blocks)
pandas_df = series.ak.to_columns(extract_all=True)
pandas_df

In [ ]:
# Example of grabbing manual information on one block/sequence

block_parser = BlockParser(20240205)
print("Sequences Run:", block_parser.getSeqNums(219))
block_parser.getBlockInfo(219,1)

In [ ]:
def day_obs_report(day_obs):
    '''
    Loop over the blocks and sequences for one day and produce a report.
    Interspace TMA events with the block info.
    '''

    block_parser = BlockParser(day_obs)
    tma_events = event_maker.getEvents(day_obs)
    blocks = block_parser.getBlockNums()

    print(f'SUMMARY REPORT FOR DAYOBS: {day_obs} \n')

    for block_id in blocks:
        sequences =  block_parser.getSeqNums(block_id)

        print(f'BLOCK:SEQ \t STATES')

        for seq_id in sequences:
            info = block_parser.getBlockInfo(block_id, seq_id)
            state_string = ' '.join([str(state) for state in info.states])
            print(f'{info.blockNumber}:{info.seqNum} \t\t {state_string}')

            # Also print any TMA events for this block/sequence
            event = block_parser.getEventsForBlock(tma_events, block_id, seq_id)
            if event: print(event)

        print(f'\n')

This HTML magick below will make it so the example report below doesn't line wrap. Each line can be quite long.

In [ ]:
%%html
<style>
div.jp-OutputArea-output pre {
    white-space: pre;
}
</style>

In [ ]:
day_obs_report(20240205)